# AgentCore Memory (단기 메모리)를 사용하는 Strands Agents - MemoryManager 사용


## 소개

이 튜토리얼은 **MemoryManager**와 **MemorySessionManager**를 사용하여 AgentCore **단기 메모리**를 갖춘 Strands agents로 **개인 어시스턴트 agent**를 구축하는 방법을 보여줍니다. Agent는 `get_last_k_turns`를 사용하여 세션의 최근 대화를 기억하고 사용자가 돌아왔을 때 원활하게 대화를 이어갈 수 있습니다.

**참고: 이것은 MemoryManager & MemorySessionManager를 사용하는 단기 메모리 샘플 버전입니다.**


### 튜토리얼 세부 정보

| 정보                | 세부 사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 단기 대화형                                                        |
| Agent 유형          | 개인 어시스턴트 Agent                                                                   |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| 튜토리얼 구성 요소 | MemoryManager를 사용한 AgentCore 단기 메모리, AgentInitializedEvent 및 MessageAddedEvent hooks   |
| 예제 복잡도  | 초급                                                                         |

학습 내용:
- MemoryManager를 사용한 대화 연속성을 위한 단기 메모리 사용
- MemorySessionManager를 사용하여 마지막 K개의 대화 턴 검색
- 실시간 정보를 위한 웹 검색 도구
- 세션 관리를 사용하여 대화 기록으로 agent 초기화
- MemoryClient에서 MemoryManager 아키텍처로 마이그레이션하는 데 사용 가능

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- AgentCore Memory 권한이 있는 AWS 자격 증명
- MemoryManager를 지원하는 Amazon Bedrock AgentCore SDK
- Amazon Bedrock models에 대한 액세스

환경 설정을 시작해봅시다!

## 단계 1: 설정 및 Import

In [2]:
!pip install -qr requirements.txt

In [3]:
import logging
from datetime import datetime
from botocore.exceptions import ClientError

# 로깅 설정 - INFO 레벨로 타임스탬프와 메시지 포맷 지정
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("personal-agent")

In [4]:
# Strands Agent 관련 모듈
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

# AgentCore 메모리 관리 모듈
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from bedrock_agentcore.memory.session import MemorySession, MemorySessionManager

# 메시지 역할 상수 정의
USER = MessageRole.USER
ASSISTANT = MessageRole.ASSISTANT

# 설정
REGION = os.getenv('AWS_REGION', 'us-east-1') # AWS 리전
ACTOR_ID = "user_123" # 고유 식별자 (Agent ID, User ID 등)
SESSION_ID = "personal_session_001" # 세션 고유 식별자

# IAM role 생성용 boto3
import boto3
import json as json_module

## 단계 2: 웹 검색 Tool

먼저 agent를 위한 간단한 웹 검색 도구를 만들어봅시다. 이것은 원래 구현에서 변경되지 않았습니다.

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        # DuckDuckGo 검색 엔진을 사용하여 웹 검색 수행
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

## 단계 3: MemoryManager를 사용하여 Memory 리소스 생성

단기 메모리의 경우 MemoryManager를 사용하여 전략 없이 메모리 리소스를 생성합니다. 이것은 `get_last_k_turns`로 검색할 수 있는 원시 대화 턴을 저장합니다.

**참고: 이 섹션은 레거시 MemoryClient 대신 MemoryManager 아키텍처를 사용합니다.**

In [ ]:
# MemoryManager 초기화
memory_manager = MemoryManager(region_name=REGION)
memory_name = "PersonalAgentMemoryManager"

logger.info(f"✅ MemoryManager initialized for region: {REGION}")
logger.info(f"Memory manager type: {type(memory_manager)}")

# MemoryManager를 사용하여 메모리 리소스 생성
logger.info(f"Creating memory '{memory_name}' for short-term conversational storage...")

try:
    memory = memory_manager.get_or_create_memory(
        name=memory_name,
        strategies=[],  # 단기 메모리는 전략 없음
        description="Short-term memory for personal agent",
        event_expiry_days=7,  # 단기 메모리 보관 기간
        memory_execution_role_arn=None,  # 단기 메모리는 선택 사항
    )
    memory_id = memory.id
    logger.info(f"✅ Successfully created/retrieved memory with MemoryManager:")
    logger.info(f"   Memory ID: {memory_id}")
    logger.info(f"   Memory Name: {memory.name}")
    logger.info(f"   Memory Status: {memory.status}")
    
except Exception as e:
    # 메모리 생성 중 오류 처리 및 상세 에러 리포팅
    logger.error(f"❌ Memory creation failed: {e}")
    logger.error(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    
    # 오류 발생 시 정리 - 부분적으로 생성된 메모리 삭제
    if 'memory_id' in locals():
        try:
            logger.info(f"Attempting cleanup of partially created memory: {memory_id}")
            memory_manager.delete_memory(memory_id)
            logger.info(f"✅ Successfully cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"❌ Failed to clean up memory: {cleanup_error}")
    
    # 원래 예외 다시 발생
    raise

## 단계 4: Session Manager 초기화

이 섹션에서는 세션 기반 메모리 작업을 위한 MemorySessionManager를 소개하고 actor 및 session을 관리하기 위한 MemorySession을 생성합니다

In [ ]:
# 세션 메모리 관리자 초기화
session_manager = MemorySessionManager(memory_id=memory.id, region_name=REGION)

# 특정 actor/session 조합을 위한 메모리 세션 생성
user_session = session_manager.create_memory_session(
    actor_id=ACTOR_ID, 
    session_id=SESSION_ID
)

logger.info(f"✅ Session manager initialized for memory: {memory.id}")
logger.info(f"✅ Memory session created for actor: {ACTOR_ID}, session: {SESSION_ID}")
logger.info(f"Session manager type: {type(session_manager)}")
logger.info(f"Memory session type: {type(user_session)}")

## 단계 5: Memory Hook Provider

이 단계에서는 MemorySession을 사용하여 메모리 작업을 자동화하는 커스텀 `MemoryHookProvider` 클래스를 정의합니다. Hook은 agent의 실행 라이프사이클의 특정 지점에서 실행되는 특수 함수입니다. 우리가 만드는 메모리 hook은 두 가지 주요 기능을 제공합니다:
1. **최근 대화 로드**: `AgentInitializedEvent` hook을 사용하여 agent가 초기화될 때 최근 대화 기록을 자동으로 로드합니다.
2. **마지막 메시지 저장**: 세션 관리자를 사용하여 새로운 대화 메시지를 저장합니다.

**MemoryClient 버전과의 주요 변경 사항:**
- MemoryClient 대신 MemorySession 사용
- 튜플 대신 ConversationalMessage 객체 사용
- create_event() 대신 add_turns() 사용
- 타입 안전성을 위한 MessageRole enum 사용

In [8]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_session: MemorySession):  # MemorySession 인스턴스 받기
        self.memory_session = memory_session
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Agent 시작 시 MemorySession을 사용하여 최근 대화 기록 로드"""
        try:
            # 사전 구성된 메모리 세션 사용 (actor_id/session_id 불필요)
            recent_turns = self.memory_session.get_last_k_turns(k=5)
            
            if recent_turns:
                # 컨텍스트를 위한 대화 기록 포맷팅
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        # EventMessage 객체와 dict 포맷 모두 처리
                        if hasattr(message, 'role') and hasattr(message, 'content'):
                            role = message['role']
                            content = message['content']
                        else:
                            role = message.get('role', 'unknown')
                            content = message.get('content', {}).get('text', '')
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Agent의 system prompt에 컨텍스트 추가
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns using MemorySession")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")

    def on_message_added(self, event: MessageAddedEvent):
        """MemorySession을 사용하여 메시지를 메모리에 저장"""
        messages = event.agent.messages
        try:
            if messages and len(messages) > 0 and messages[-1]["content"][0].get("text"):
                message_text = messages[-1]["content"][0]["text"]
                message_role = MessageRole.USER if messages[-1]["role"] == "user" else MessageRole.ASSISTANT
                
                # 메모리 세션 인스턴스 사용 (actor_id/session_id 전달 불필요)
                result = self.memory_session.add_turns(
                    messages=[ConversationalMessage(message_text, message_role)]
                )
                
                event_id = result['eventId']
                logger.info(f"✅ Stored message with Event ID: {event_id}, Role: {message_role.value}")
                
        except Exception as e:
            logger.error(f"Memory save error: {e}")
            import traceback
            logger.error(f"Full traceback: {traceback.format_exc()}")
    
    def register_hooks(self, registry: HookRegistry):
        # 메모리 hook 등록
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        logger.info("✅ Memory hooks registered with MemorySession")


## 단계 6: 웹 검색 기능을 갖춘 개인 어시스턴트 Agent 생성

이 agent는 MemorySessionManager에서 생성된 MemorySession과 함께 작동하는 MemoryHookProvider를 사용합니다.

In [ ]:
def create_personal_agent():
    """MemorySession을 사용하여 메모리와 웹 검색 기능을 갖춘 개인 agent 생성"""
    agent = Agent(
        name="PersonalAssistant",
        model="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # 또는 선호하는 모델
        system_prompt=f"""You are a helpful personal assistant with web search capabilities.
        
        You can help with:
        - General questions and information lookup
        - Web searches for current information
        - Personal task management
        
        When you need current information, use the websearch function.
        Today's date: {datetime.today().strftime('%Y-%m-%d')}
        Be friendly and professional.""",
        hooks=[MemoryHookProvider(user_session)],  # MemorySession을 hook에 전달
        tools=[websearch],
    )
    return agent

# Agent 생성
agent = create_personal_agent()
logger.info("✅ Personal agent created with MemorySession and web search")

#### 축하합니다! MemoryManager & MemorySession을 사용하는 agent가 준비되었습니다
## Agent를 테스트해봅시다

In [ ]:
# Test conversation with memory
print("=== First Conversation ===")
print(f"User: My name is Alex and I'm interested in learning about AI.")
print(f"Agent: ", end="")
agent("My name is Alex and I'm interested in learning about AI.")

In [ ]:
print(f"User: Can you search for the latest AI trends in 2025?")
print(f"Agent: ", end="")
agent("Can you search for the latest AI trends in 2025?")

In [ ]:
print(f"User: I'm particularly interested in machine learning applications.")
print(f"Agent: ", end="")
agent("I'm particularly interested in machine learning applications.")

## MemorySessionManager를 사용한 메모리 연속성 테스트

메모리 시스템이 올바르게 작동하는지 테스트하기 위해 agent의 새 인스턴스를 생성하고 MemorySessionManager를 사용하여 이전에 저장된 정보에 액세스할 수 있는지 확인합니다:

In [ ]:
# Agent의 새 인스턴스 생성 (사용자가 돌아온 것을 시뮬레이션)
print("=== User Returns - New Session ===")
new_agent = create_personal_agent()

# 메모리 연속성 테스트
print(f"User: What was my name again?")
print(f"Agent: ", end="")
new_agent("What was my name again?")

print(f"User: Can you search for more information about machine learning?")
print(f"Agent: ", end="")
new_agent("Can you search for more information about machine learning?")

## MemorySession을 사용하여 저장된 메모리 보기

In [ ]:
# MemorySession을 사용하여 메모리에 저장된 내용 확인
print("=== Memory Contents ===")
recent_turns = user_session.get_last_k_turns(k=3) 

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

## 요약

이 튜토리얼은 MemorySessionManager와 MemorySession을 모두 사용하여 개인 어시스턴트 agent를 구축하는 방법을 보여주었습니다. 학습한 내용:

- **MemorySessionManager**: 여러 세션에 걸친 메모리 작업을 위한 고수준 관리자
- **MemorySession**: 반복적인 매개변수 전달을 제거하는 세션별 인터페이스. MemorySession을 사용하면 모든 메서드에 actor_id/session_id를 전달할 필요가 없습니다
- **타입 안전성**: Session은 생성 시 특정 actor/session에 바인딩됩니다
- **더 나은 캡슐화**: 세션별 작업이 세션 객체 내에 포함됩니다
- **Memory Hooks**: Agent hook은 세션 기반 아키텍처와 함께 작동할 수 있습니다
- **대화 연속성**: MemoryManager & MemorySession을 사용한 단기 메모리 기능 유지

### MemorySession의 주요 이점:
1. **간소화된 API**: 모든 메서드 호출에 actor_id/session_id를 전달할 필요가 없습니다
2. **사전 구성된 컨텍스트**: Session은 생성 시 특정 actor/session에 바인딩됩니다
3. **일관된 인터페이스**: 모든 세션 작업은 동일한 사전 구성된 컨텍스트를 사용합니다


## 정리 (선택 사항)

In [ ]:
# MemoryManager를 사용하여 메모리 리소스 삭제하려면 주석 해제
# try:
#     memory_manager.delete_memory(memory_id)
#     logger.info(f"✅ Deleted memory: {memory_id}")
# except Exception as e:
#     logger.error(f"Failed to delete memory: {e}")